In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vispol/dhaka-streets-coco/README.roboflow.txt
/kaggle/input/vispol/dhaka-streets-coco/valid/towers-22_jpg.rf.f54947d6c80fc8d47182fd9788a4ac57.jpg
/kaggle/input/vispol/dhaka-streets-coco/valid/towers-50_jpg.rf.49394d9e860ba922cb9ae283f37b2208.jpg
/kaggle/input/vispol/dhaka-streets-coco/valid/constructionMat-44_jpg.rf.eff7fb91a83ff2bd1e933a9614573601.jpg
/kaggle/input/vispol/dhaka-streets-coco/valid/billboard-33_jpg.rf.54968ab5e6443f93bb889a697c97e0d0.jpg
/kaggle/input/vispol/dhaka-streets-coco/valid/wires-108_jpg.rf.24070a9a0e7e5d2e6402e96941219074.jpg
/kaggle/input/vispol/dhaka-streets-coco/valid/streetLitters2-223_jpg.rf.093229e3906161a44a8718bc1be59c8b.jpg
/kaggle/input/vispol/dhaka-streets-coco/valid/wires-187_jpg.rf.73011466db457a9f611199b78d32bcfa.jpg
/kaggle/input/vispol/dhaka-streets-coco/valid/billboard-79_jpg.rf.346c0495a95fb9a994b62b758d50542d.jpg
/kaggle/input/vispol/dhaka-streets-coco/valid/constructionMat-20_jpg.rf.dc3657d5adf95206f24754a2f5908dad.jpg
/kaggle

In [2]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=2dd7f2c82573dc0029f0b7a45a50089aa28e823cd76812ec1004853eae4c4a19
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
pytorch-lightning 1.7.7 requires PyYAML>=5.4, but you have pyyaml 5.1 which is incompatible.
pandas-profiling 3.1.0 requires markupsafe~=2.0.1, but you have markupsafe 2.1.1 which is incompat

In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
torch:  1.11 ; cuda:  1.11.0
detectron2: 0.6


In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [5]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("my_dataset_train", {}, "/kaggle/input/vispol/dhaka-streets-coco/train/_annotations.coco.json", "/kaggle/input/vispol/dhaka-streets-coco/train")
register_coco_instances("my_dataset_val", {}, "/kaggle/input/vispol/dhaka-streets-coco/valid/_annotations.coco.json", "/kaggle/input/vispol/dhaka-streets-coco/valid")

In [6]:
#visualize training data
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

WARNING [12/06 14:51:49 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/06 14:51:49 d2.data.datasets.coco]: Loaded 1117 images in COCO format from /kaggle/input/vispol/dhaka-streets-coco/train/_annotations.coco.json


In [7]:
my_dataset_train_metadata

namespace(name='my_dataset_train',
          json_file='/kaggle/input/vispol/dhaka-streets-coco/train/_annotations.coco.json',
          image_root='/kaggle/input/vispol/dhaka-streets-coco/train',
          evaluator_type='coco',
          thing_classes=['pollutants', '0', '1', '2', '3', '4', '5'],
          thing_dataset_id_to_contiguous_id={0: 0,
                                             1: 1,
                                             2: 2,
                                             3: 3,
                                             4: 4,
                                             5: 5,
                                             6: 6})

In [8]:
dataset_dicts

[{'file_name': '/kaggle/input/vispol/dhaka-streets-coco/train/towers-200_jpg.rf.0091f429b7e095410054c79a32f2519e.jpg',
  'height': 500,
  'width': 500,
  'image_id': 0,
  'annotations': [{'iscrowd': 0,
    'bbox': [44, 149, 153, 305],
    'category_id': 5,
    'bbox_mode': <BoxMode.XYWH_ABS: 1>}]},
 {'file_name': '/kaggle/input/vispol/dhaka-streets-coco/train/towers-47_jpg.rf.02808f4344303abbb8a0ec8a21c7871f.jpg',
  'height': 500,
  'width': 500,
  'image_id': 1,
  'annotations': [{'iscrowd': 0,
    'bbox': [86, 147, 71, 211],
    'category_id': 5,
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'iscrowd': 0,
    'bbox': [253, 161, 39, 161],
    'category_id': 5,
    'bbox_mode': <BoxMode.XYWH_ABS: 1>}]},
 {'file_name': '/kaggle/input/vispol/dhaka-streets-coco/train/billboard-143_jpg.rf.00dfab6f067ef969545546d6f4b9fa6d.jpg',
  'height': 500,
  'width': 500,
  'image_id': 2,
  'annotations': [{'iscrowd': 0,
    'bbox': [87, 100, 161, 151],
    'category_id': 1,
    'bbox_mode': <BoxMode.XY

In [9]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [10]:
from detectron2.engine import DefaultTrainer


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ('my_dataset_val',)

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR =0.00025  


cfg.SOLVER.MAX_ITER = 14000 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = [] 

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7


cfg.TEST.EVAL_PERIOD = 140
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()



[12/06 14:51:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f6e8b1.pkl: 243MB [00:03, 70.3MB/s]                              


[12/06 14:52:04 d2.engine.train_loop]: Starting training from iteration 0


/opt/conda/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:2227.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/06 14:52:34 d2.utils.events]:  eta: 4:19:46  iter: 19  total_loss: 2.332  loss_cls: 2.082  loss_box_reg: 0.1053  loss_rpn_cls: 0.1216  loss_rpn_loc: 0.01325  time: 1.1131  data_time: 0.1109  lr: 4.9953e-06  max_mem: 8018M
[12/06 14:52:56 d2.utils.events]:  eta: 4:21:29  iter: 39  total_loss: 2.254  loss_cls: 1.959  loss_box_reg: 0.1218  loss_rpn_cls: 0.1058  loss_rpn_loc: 0.01398  time: 1.1184  data_time: 0.1016  lr: 9.9903e-06  max_mem: 8018M
[12/06 14:53:18 d2.utils.events]:  eta: 4:21:10  iter: 59  total_loss: 1.902  loss_cls: 1.694  loss_box_reg: 0.09777  loss_rpn_cls: 0.1055  loss_rpn_loc: 0.01259  time: 1.1132  data_time: 0.0921  lr: 1.4985e-05  max_mem: 8018M
[12/06 14:53:40 d2.utils.events]:  eta: 4:21:12  iter: 79  total_loss: 1.477  loss_cls: 1.298  loss_box_reg: 0.09696  loss_rpn_cls: 0.09768  loss_rpn_loc: 0.01176  time: 1.1138  data_time: 0.0949  lr: 1.998e-05  max_mem: 8018M
[12/06 14:54:02 d2.utils.events]:  eta: 4:20:31  iter: 99  total_loss: 1.134  loss_cls: 0.9149